In [6]:
from typing import List
from pydantic import BaseModel
import speech_recognition as sr
from fastapi import FastAPI, File, UploadFile, HTTPException, Form
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import uvicorn
import asyncio
import pytesseract
import os
from pdf2image import convert_from_path
import json

r = sr.Recognizer()
app = FastAPI()

origins = [
    "http://localhost:3000",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # Cho phép các nguồn gốc được xác định
    allow_credentials=True,
    allow_methods=["*"],  # Cho phép tất cả các phương thức (GET, POST, PUT, DELETE)
    allow_headers=["*"],  # Cho phép tất cả các headers
)

@app.get("/")
async def read_root():
    return {"Hello": "World"}

@app.post("/login")
async def login(body: dict):
    users = json.load(open("users.json", "r"))

    username = body.get("username")
    password = body.get("password")

    if username not in [user["username"] for user in users]:
        return JSONResponse(status_code=404, content={"message": "User not found"})
    user = [user for user in users if user["username"] == username][0]
    if user["password"] != password:
        return JSONResponse(status_code=401, content={"message": "Incorrect password"})
    return {"status": "success"}

@app.post("/register")
async def register(body: dict):
    users = json.load(open("users.json", "r"))

    username = body.get("username")
    password = body.get("password")
    print(username, password)

    if username in [user["username"] for user in users]:
        return JSONResponse(status_code=400, content={"message": "User already exists"})
    users.append({"username": username, "password": password})
    json.dump(users, open("users.json", "w"))
    return {"status": "success"}



@app.post("/convertAudio")
async def convert_audio_to_text(file: UploadFile = File(...), language: str = Form(...)):
    if not os.path.exists("./audio"):
        os.makedirs("./audio")
    
    lang = language
    with open(f"./audio/{file.filename.split('.')[0]}.wav", "wb") as audio:
        audio.write(file.file.read())
    with sr.AudioFile("./audio/" + file.filename.split('.')[0] + ".wav") as source:
        audio = r.record(source)
    try:
        text = r.recognize_google(audio, language=lang)
    except sr.UnknownValueError:
        text = "Could not understand the audio"
    except sr.RequestError as e:
        text = "Could not request results; {0}".format(e)
    return {"text": text}

@app.post("/getImg")
async def get_img(file: UploadFile = File(...)):
    if not os.path.exists("./img"):
        os.makedirs("./img")

    with open(f"./img/{file.filename}", "wb") as img:
        img.write(file.file.read())
    return True


class ImageNames(BaseModel):
    names: List[str]
@app.post("/convertImg")
async def convert_img_to_text(image_names: ImageNames):
    results = {}
    print(image_names.names)
    for name in image_names.names:
        if name.endswith(".pdf"):
            pdf_path = f"./img/{name}"
            if not os.path.exists(pdf_path):
                raise HTTPException(status_code=404, detail=f"PDF {name} not found")
            images_pdf = convert_from_path(pdf_path, poppler_path="C:/games/poppler-24.02.0/Library/bin")
            text = ""
            for img in images_pdf:
                text += pytesseract.image_to_string(img)
            results[name] = text
            
        else:
            img_path = f"./img/{name}"
            if not os.path.exists(img_path):
                raise HTTPException(status_code=404, detail=f"Image {name} not found")
            text = pytesseract.image_to_string(img_path)
            results[name] = text

    return {"results": "OK"}
        
if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [20456]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56887 - "OPTIONS /register HTTP/1.1" 200 OK
hieu 123
INFO:     127.0.0.1:56887 - "POST /register HTTP/1.1" 200 OK
INFO:     127.0.0.1:56899 - "OPTIONS /login HTTP/1.1" 200 OK
INFO:     127.0.0.1:56899 - "POST /login HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [20456]
